In [1]:
!pip install merlin-core

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/11

In [5]:
!pip install nvtabular

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for merlin-dataloader: filename=merlin_dataloader-23.2.1-py3-none-any.whl size=40725 sha256=8c1ce6ff66213d14d0e396139c2f781dafee91640123865e26167e3afb9ef805
  Stored in directory: /root/.cache/pip/wheels/34/bb/93/d35bf8e031723b123fc587e5b201e769d6398f30f2d746f17c
Successfully built merlin-dataloader


In [6]:
!pip install merlin-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.1/392.1 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for merlin-models: filename=merlin_models-23.2.0-py3-none-any.whl size=371141 sha256=fce669fc10d21cb3cb4f4eea554cc01a9f2d9bd10ef114593b06c14f2adfeac4
  Stored in directory: /root/.cache/pip/wheels/af/60/7f/5dd12f4c95d06237dc60140170042f2cac4972320b2cbdda5a
Successfully built merlin-models


In [2]:
!pip install redis==4.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 kB 5.7 MB/s eta 0:00:00


### Importing Libraries

In [7]:
import os
import logging
import time
import warnings
warnings.filterwarnings('ignore')

import nvtabular as nvt
import merlin.models.tf as mm
import tensorflow as tf

from nvtabular.ops import *

from merlin.datasets.synthetic import generate_data
from merlin.datasets.ecommerce import transform_aliccp
from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.models.utils.dataset import unique_rows_by_features
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset


# disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING)

### Generate and Process Synthetic Ali-CCP Dataset

In [8]:
# Generate the synthetic data
NUM_ROWS = 1000000
TRAIN_SIZE = 0.7
VALID_SIZE = 0.3

train, valid = generate_data("aliccp-raw", NUM_ROWS, set_sizes=(TRAIN_SIZE, VALID_SIZE))

In [9]:
# Define output path for data
DATA_DIR = "/model-data/aliccp"
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "processed")
OUTPUT_RETRIEVAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, "retrieval")
CATEGORY_TEMP_DIR = os.path.join(DATA_DIR, "categories")

In [10]:
# Define NVTabular Feature Transformation Pipeline

user_id_raw = ["user_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsUserFeatures()
item_id_raw = ["item_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsItemFeatures()

user_id = ["user_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemID()

item_features = (
    ["item_category", "item_shop", "item_brand"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemFeatures()
)

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR)
    >> TagAsUserFeatures()
)

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id + item_id + item_features + user_features +  user_id_raw + item_id_raw + targets

# add dropna op to filter rows with nulls
outputs = outputs >> Dropna()

In [11]:
# Transform data and create files
transform_aliccp((train, valid), OUTPUT_DATA_DIR, nvt_workflow=outputs)

### Feature Engineering for Candidate Retrieval

In [12]:
# Load Datasets from generated files
train_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "train", "*.parquet"))
valid_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "valid", "*.parquet"))

In [13]:
# Define NVTabular Feature Transformation Pipeline

inputs = train_retrieval.schema.column_names

# Select only positive interaction rows where click==1 in the dataset with Filter() operator
outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

# Execute the transformation workflow for both datasets
workflow = nvt.Workflow(outputs)
workflow.fit(train_retrieval)
workflow.transform(train_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train")
)
workflow.transform(valid_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid")
)

In [14]:
# Read transformed parquet files as Dataset objects
train_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train", "*.parquet"), part_size="500MB")
valid_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid", "*.parquet"), part_size="500MB")

In [15]:
# Create model input schema
schema = train_retrieval.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['user_id_raw', 'item_id_raw', 'click'])
train_retrieval.schema = schema
valid_retrieval.schema = schema

In [16]:
schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.num_buckets,properties.freq_threshold,properties.start_index,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.CATEGORICAL, Tags.ID, Tags.USER_ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,780.0,67.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,779,user_id
1,item_id,"(Tags.ITEM_ID, Tags.CATEGORICAL, Tags.ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_id
2,item_category,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_category
3,item_shop,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_shop
4,item_brand,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_brand
5,user_shops,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,780.0,67.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,779,user_shops
6,user_profile,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,47.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,46,user_profile
7,user_group,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,11.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,10,user_group
8,user_gender,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,3.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,2,user_gender
9,user_age,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,7.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,6,user_age


#### Designing Retrieval Model Architecture

In [17]:
# Function to create the two tower retrieval model

def create_two_tower(tower_dim: int, encoder_dim: int, optimizer: str, k: int, tags) -> mm.TwoTowerModelV2:
    # User/Query Tower
    user_schema = schema.select_by_tag(tags.USER)
    # create user (query) tower input block
    user_inputs = mm.InputBlockV2(user_schema)
    # create user (query) encoder block
    query = mm.Encoder(
        user_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )

    # Item/Candidate Tower
    item_schema = schema.select_by_tag(tags.ITEM)
    # create item (candidate) tower input block
    item_inputs = mm.InputBlockV2(item_schema)
    # create item (candidate) encoder block
    candidate = mm.Encoder(
        item_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )
    
    # Build Model Class
    model = mm.TwoTowerModelV2(query, candidate)
    model.compile(
        optimizer=optimizer,
        run_eagerly=False,
        loss="categorical_crossentropy",
        metrics=[mm.RecallAt(k), mm.NDCGAt(k)]
    )
    return model

In [18]:
# Initialize model
retrieval_model = create_two_tower(
    tower_dim=64,
    encoder_dim=128,
    optimizer="adam",
    k=10,
    tags=Tags
)

# Fit model
retrieval_model.fit(train_retrieval, validation_data=valid_retrieval, batch_size=4096, epochs=2)

Epoch 1/2
86/86 [==============================] - 130s 1s/step - loss: 8.2961 - recall_at_10: 0.0260 - ndcg_at_10: 0.0239 - regularization_loss: 0.0000e+00 - loss_batch: 8.2890 - val_loss: 8.2935 - val_recall_at_10: 0.0300 - val_ndcg_at_10: 0.0299 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 8.0110
Epoch 2/2
86/86 [==============================] - 116s 1s/step - loss: 8.2956 - recall_at_10: 0.0289 - ndcg_at_10: 0.0286 - regularization_loss: 0.0000e+00 - loss_batch: 8.2884 - val_loss: 8.2939 - val_recall_at_10: 0.0213 - val_ndcg_at_10: 0.0196 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 8.0112


#### Evaluate the model accuracy

In [19]:
# Create candidate/item features for evaluation
candidate_features = unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID)

# Here's a sneek peek of the item features data
candidate_features.head()

,item_id,item_category,item_shop,item_brand
0,11,11,11,11
1,1,1,1,1
2,15,15,15,15
3,9,9,9,9
4,29,29,29,29


In [20]:
# Convert model to a top_k_encoder
topk_model = retrieval_model.to_top_k_encoder(candidate_features, k=20, batch_size=128)
topk_model.compile(run_eagerly=False)

In [21]:
# Create data loader for validation data
eval_loader = mm.Loader(valid_retrieval, batch_size=1024).map(mm.ToTarget(schema, "item_id"))

# Evaluation
metrics = topk_model.evaluate(eval_loader, return_dict=True)
metrics

147/147 [==============================] - 8s 32ms/step - loss: 0.4076 - recall_at_10: 0.0843 - mrr_at_10: 0.0330 - ndcg_at_10: 0.0450 - map_at_10: 0.0330 - precision_at_10: 0.0084 - regularization_loss: 0.0000e+00 - loss_batch: 0.4078


{'loss': 0.4076088070869446,
 'recall_at_10': 0.08425066620111465,
 'mrr_at_10': 0.03302222117781639,
 'ndcg_at_10': 0.04498661309480667,
 'map_at_10': 0.03302222117781639,
 'precision_at_10': 0.008425066247582436,
 'regularization_loss': 0.0,
 'loss_batch': 0.4356307089328766}

#### Generate top-K candidates

In [22]:
# Create query/user features for evaluation
user_features = unique_rows_by_features(valid_retrieval, Tags.USER, Tags.USER_ID)
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,9,9,1,1,1,1,1,1,1,9,9,9
1,13,13,1,1,1,1,1,1,1,13,13,13
2,51,51,2,1,1,1,1,1,1,51,51,51
3,22,22,1,1,1,1,1,1,1,22,22,22
4,4,4,1,1,1,1,1,1,1,4,4,4


In [23]:
# Check out one batch of 8 Users
loader = mm.Loader(user_features, batch_size=8, shuffle=False)
batch = next(iter(loader))
print(batch[0]['user_id'])

tf.Tensor(
[[ 9]
 [13]
 [51]
 [22]
 [ 4]
 [74]
 [47]
 [14]], shape=(8, 1), dtype=int32)


In [24]:
scores, reccommended_item_ids = topk_model(batch[0])

In [25]:
# Recommended Items for the batch of 8 Users
reccommended_item_ids

<tf.Tensor: shape=(8, 20), dtype=int32, numpy=
array([[ 16,  47,  83,  31,   7,  18,  80, 754, 376, 180, 140,  35,  72,
        385, 344, 483,   6, 217,  34, 201],
       [ 30,  42,   7, 234,  36, 353, 367, 687,  18, 359, 351,  69,   5,
        427, 392, 105, 343,   6, 445, 337],
       [ 10,  16, 180, 137, 261, 285, 186, 218, 174, 286,  34, 385, 324,
        366, 182,  62,   4, 275, 338, 193],
       [116,   2,  22, 302, 468,   3,  17, 370, 216, 357, 178, 259, 236,
        153, 515, 305, 168,  25,  87,  58],
       [ 16,  10, 445,  47,   7, 261, 391, 483, 217, 496, 180, 428, 134,
        104, 323,  80, 498, 329,  14,  45],
       [216,  22, 112, 168, 116, 219,  23, 251,  25,  32,  24, 399, 739,
        289,  17, 114,  43, 295, 117, 166],
       [ 32,   4, 739,  51,  15,   9, 199, 216, 302, 347, 336,  30,  84,
         27, 246, 327, 628, 123,  70, 517],
       [ 18,  30,   4, 359, 344,  34, 286, 234, 161, 385, 201, 552,   6,
        376, 256,   3, 673,  73, 628, 586]], dtype=int32)>

In [26]:
# Recommendation "scores" (higher is better) for each recommended item for the batch of 8 Users
scores

<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0.02237884, 0.01794879, 0.0142032 , 0.0141079 , 0.01247173,
        0.01175879, 0.00937201, 0.0091588 , 0.00894753, 0.00754664,
        0.00592429, 0.00535357, 0.00524596, 0.00498738, 0.00455313,
        0.00451372, 0.00422954, 0.00191583, 0.00171764, 0.00161741],
       [0.05091146, 0.02425309, 0.02207669, 0.01910865, 0.01834677,
        0.01747436, 0.01416761, 0.01329941, 0.0125213 , 0.01102724,
        0.00879338, 0.00871183, 0.00794389, 0.00545314, 0.0053711 ,
        0.00441133, 0.0035553 , 0.00214313, 0.00174285, 0.00166852],
       [0.06806048, 0.06215564, 0.05406964, 0.05040586, 0.04563279,
        0.04525307, 0.04105154, 0.03984092, 0.03962542, 0.0393935 ,
        0.03781381, 0.03731435, 0.03505041, 0.03426677, 0.03385994,
        0.0331247 , 0.03239864, 0.03140127, 0.03095391, 0.03073781],
       [0.03377665, 0.03072289, 0.02960858, 0.02510477, 0.02425186,
        0.022434  , 0.02121826, 0.02066939, 0.02032492, 0.019291

### Ranking Model